## Imports 

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


## Read Data

In [2]:
data_file_path = 'Dataset.csv'
movie_data = pd.read_csv(data_file_path)
#movie_data.shape
movie_data.describe()

,Unnamed: 0,vote_count,vote_average,duration,revenue,popularity,id,financial_investment
count,4803.000000,4803.000000,4323.000000,4801.000000,4.755000e+03,4803.000000,4803.000000,4.419000e+03
mean,2401.000000,690.217989,6.090354,106.875859,8.231486e+07,21.492301,57165.484281,2.898466e+07
std,1386.651002,1234.585891,1.193315,22.611935,1.630872e+08,31.816650,88694.614033,4.065526e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.000000,0.000000e+00
25%,1200.500000,54.000000,5.600000,94.000000,0.000000e+00,4.668070,9014.500000,7.500000e+05
50%,2401.000000,235.000000,6.200000,103.000000,1.918402e+07,12.921594,14629.000000,1.480000e+07
75%,3601.500000,737.000000,6.800000,118.000000,9.311911e+07,28.313505,58610.500000,4.000000e+07
max,4802.000000,13752.000000,10.000000,338.000000,2.787965e+09,875.581305,459488.000000,3.800000e+08


In [3]:
movie_data.columns

y = movie_data.revenue

In [4]:
movie_data.drop(['revenue'], axis = 1, inplace=True)
movie_data.head()

,Unnamed: 0,vote_count,vote_average,title,tagline,status,spoken_languages,duration,release,countries_of_production,production,popularity,overview,original_title,original_language,keywords,id,homepage,genres,financial_investment
0,0,11800,7.2,Avatar,Enter the World of Pandora.,Released,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",162.0,2009-12-10,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",150.437577,"In the 22nd century, a paraplegic Marine is di...",Avatar,en,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",19995,http://www.avatarmovie.com/,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",237000000.0
1,1,4500,6.9,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.",Released,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",169.0,2007-05-19,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",139.082615,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,en,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",285,http://disney.go.com/disneypictures/pirates/,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",300000000.0
2,2,4466,6.3,Spectre,A Plan No One Escapes,Released,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",148.0,2015-10-26,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",107.376788,A cryptic message from Bond’s past sends him o...,Spectre,en,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",206647,http://www.sonypictures.com/movies/spectre/,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",245000000.0
3,3,9106,7.6,The Dark Knight Rises,The Legend Ends,Released,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",165.0,2012-07-16,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",112.312950,Following the death of District Attorney Harve...,The Dark Knight Rises,en,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",49026,http://www.thedarkknightrises.com/,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",250000000.0
4,4,2124,6.1,John Carter,"Lost in our world, found in another.",Released,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",132.0,2012-03-07,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",43.926995,"John Carter is a war-weary, former military ca...",John Carter,en,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",49529,http://movies.disney.com/john-carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",260000000.0


In [5]:
movie_features = ['vote_count', 'vote_average', 'duration',  'popularity', 'financial_investment']
X = movie_data[movie_features]

X.head()

,vote_count,vote_average,duration,popularity,financial_investment
0,11800,7.2,162.0,150.437577,237000000.0
1,4500,6.9,169.0,139.082615,300000000.0
2,4466,6.3,148.0,107.376788,245000000.0
3,9106,7.6,165.0,112.312950,250000000.0
4,2124,6.1,132.0,43.926995,260000000.0


## Prediction 

### rmsle pour mesurer performance model pour pas que favoriser les gros blockbuster
#### Formule a faire:
#### Formule : racine((somme(log(T)-log(P))²)/n)

In [6]:
X.head()

,vote_count,vote_average,duration,popularity,financial_investment
0,11800,7.2,162.0,150.437577,237000000.0
1,4500,6.9,169.0,139.082615,300000000.0
2,4466,6.3,148.0,107.376788,245000000.0
3,9106,7.6,165.0,112.312950,250000000.0
4,2124,6.1,132.0,43.926995,260000000.0


In [7]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
movie_model = DecisionTreeRegressor(random_state=1)

# Fit model
movie_model.fit(X, y)
DecisionTreeRegressor(random_state=1)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
print("Making prediction for following 5 movies")
print(X.head())
print("The predictions are")
print(movie_model.predict(X.head()))

Making prediction for following 5 movies
   vote_count  vote_average  duration  popularity  financial_investment
0       11800           7.2     162.0  150.437577           237000000.0
1        4500           6.9     169.0  139.082615           300000000.0
2        4466           6.3     148.0  107.376788           245000000.0
3        9106           7.6     165.0  112.312950           250000000.0
4        2124           6.1     132.0   43.926995           260000000.0
The predictions are
[2.78796509e+09 9.61000000e+08 8.80674609e+08 1.08493910e+09
 2.84139100e+08]
